# Model has been in production for one month
We collected the production data, but we do not have the ground truth available yet

In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from titanic_pipeline.config import config
import pandas as pd
import joblib
from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import (
    DataDriftTab,
    DataQualityTab
)
from evidently import ColumnMapping
raw_prod = pd.read_csv(config.DATASETS_DIR + 'prod_data.csv')

column_mapping = ColumnMapping(target='survived', prediction='prediction')
model = joblib.load(config.PRODUCTION_MODEL_FILE)

raw_train = pd.read_csv(config.DATASET_FILE)
target_train = raw_train[[config.TARGET]]
raw_train = raw_train.drop(columns=config.TARGET)

dashboard = Dashboard(tabs=[DataQualityTab(), DataDriftTab()])
dashboard.calculate(raw_train, raw_prod, column_mapping=column_mapping)
dashboard.save("../reports/data_quality_drift_raw.html")

/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning: divide by zero encountered in true_divide
  terms = (f_obs_float - f_exp)**2 / f_exp
/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning: divide by zero encountered in true_divide
  terms = (f_obs_float - f_exp)**2 / f_exp
/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning: divide by zero encountered in true_divide
  terms = (f_obs_float - f_exp)**2 / f_exp
/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning: divide by zero encountered in true_divide
  terms = (f_obs_float - f_exp)**2 / f_exp
/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning: divide by zero encountered in true_divide
  terms = (f_obs_float - f_exp)**2 / f_exp
/Users/andre.bajorat/tec/.tita

In [2]:
processor = model['preprocessor']
processed_prod = pd.DataFrame(processor.transform(raw_prod.copy()), columns=processor.get_feature_names_out())
processed_train = pd.DataFrame(processor.transform(raw_train.copy()), columns=processor.get_feature_names_out())

processed_prod['prediction'] = model.predict(raw_prod.copy())
processed_train['prediction'] = model.predict(raw_train.copy())

data_drift_report = Dashboard(tabs=[DataQualityTab(), DataDriftTab()])
data_drift_report.calculate(processed_train, processed_prod, column_mapping = column_mapping)
data_drift_report.save("../reports/data_quality_drift_processed.html")

## Release of ground truth data for month January

In [3]:
from evidently.dashboard.tabs import CatTargetDriftTab

target_prod = pd.read_csv(config.DATASETS_DIR + 'prod_truth.csv')

processed_prod[config.TARGET] = target_prod[config.TARGET]
processed_train[config.TARGET] = target_train[config.TARGET]
data_and_target_drift_report = Dashboard(tabs=[DataDriftTab(), CatTargetDriftTab()])
data_and_target_drift_report.calculate(processed_train, processed_prod, column_mapping = column_mapping)
data_and_target_drift_report.save("../reports/data_n_concept_drift.html")

In [4]:
from evidently.dashboard.tabs import ClassificationPerformanceTab

processed_prod[config.TARGET] = target_prod[config.TARGET]
rf_model = joblib.load(config.BASE_DIR +"/models/RandomForest_Best.sav")
rf_prod = processed_prod.copy()
rf_prod['prediction'] = rf_model.predict(raw_prod.copy())

dashboard = Dashboard(tabs=[ClassificationPerformanceTab(verbose_level=1)])
dashboard.calculate(reference_data=processed_prod, current_data=rf_prod, column_mapping = column_mapping)
dashboard.save('../reports/regression_report.html')

In [5]:
new_train_data = raw_prod.copy()
new_train_data[config.TARGET] = target_prod[config.TARGET]
old_train_data = raw_train.copy()
new_train_data[config.TARGET] = target_train[config.TARGET]

new_train_data = pd.concat([old_train_data, new_train_data])
new_train_data.to_csv(config.DATASETS_DIR +"/prod_plus_old_data.csv")

In [4]:
import requests
import json

url = "http://127.0.0.1:8000/prediction"

payload = json.dumps({
  "pclass": 1,
  "name": "Allen, Miss. Elisabeth Walton",
  "sex": "female",
  "age": "29",
  "sibsp": 0,
  "parch": 0,
  "ticket": "24160",
  "fare": "211.3375",
  "cabin": "B5",
  "embarked": "S",
  "boat": "2",
  "body": "?"
})
headers = {
  'Content-Type': 'application/json'
}

In [6]:
raw_prod = pd.read_csv(config.DATASETS_DIR + 'prod_data.csv')

for record in json.loads(raw_prod.to_json(orient='records')):
    response = requests.request("POST", url, headers=headers, data=json.dumps(record))
    print(response)
    


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [7]:
import pandas as pd
import sys
import os
import json

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from titanic_pipeline.config import config
inputs = []
preds = []

with open(f"{config.BASE_DIR}/prediction_api.log") as f:
    for line in f:
        if "Prediction:" in line:
            payload = json.loads(line.split('Prediction: ')[1])
            inputs.append(payload['input_raw'])
            preds.append(payload['prediction'])

In [8]:
prod = pd.DataFrame(inputs)
prod['prediction'] = preds

In [9]:
prod

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,prediction
0,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,1
1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,1
2,3,"Rintamaki, Mr. Matti",male,35,0,0,STON/O 2. 3101273,7.125,?,S,?,?,0
3,3,"Nakid, Mr. Sahid",male,20,1,1,2653,15.7417,?,C,C,?,0
4,3,"Lyntakoff, Mr. Stanko",male,?,0,0,349235,7.8958,?,S,?,?,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,3,"Buckley, Mr. Daniel",male,21,0,0,330920,7.8208,?,Q,13,?,0
653,3,"Yousif, Mr. Wazli",male,?,0,0,2647,7.225,?,C,?,?,0
654,1,"Andrews, Mr. Thomas Jr",male,39,0,0,112050,0.0,A36,S,?,?,0
655,3,"Flynn, Mr. John",male,?,0,0,368323,6.95,?,Q,?,?,0
